Setup code

In [49]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY in .env file
# UPSTAGE_API_KEY=your_api_key

import warnings
warnings.filterwarnings("ignore")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [50]:
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

query = f"""
Make 3 questions using this text.

{text}
"""

query2 = """
Linear transformation is a function that satisfies two features.
One is homogenity, and other is additivity.

Linear transformation of the subspace is also subspace.
Prove or disprove this statement based on given contents.
"""

query3 = """
The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""

query4 = """
What is a Mahjong game?
"""

In [51]:
from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
In this query, separate the instruction and context.
Do not answer to the query, just separate it. Also, only use given query itself, do not generate or change any words.
Instruction is saying about what AI should do.
Context is can be a base information or some text to convert or manipulate.
Context can be not provided(N/A for this case).
Instruction and context must not have duplicate contents.
Print instruction as first paragraph and context as second paragraph.
---
Query : {query}
"""
)
chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"query": query2})

instr, context = [s.strip() for s in answer.split("Instruction:")[1].split("Context:")]
print(instr)
print(context)

The instruction is asking the AI to prove or disprove the statement "Linear transformation of the subspace is also subspace" based on the given contents.
The context provides information about linear transformation and its features. It states that linear transformation is a function that satisfies two features: homogeneity and additivity.

---


In [52]:
# 각 chain을 실행하고 결과 누적으로 얻음
# (전체 히스토리 누적 후) 최종 결과 질문

prompt_template = PromptTemplate.from_template(
    """
For instruction "{instr}", give me what to do to achieve this instruction for AI.
Only print 1., 2., 3., ... steps to do. Do not answer to the question, and only print steps.
Each step should be a one-liner.
"""
)

chain = prompt_template | llm | StrOutputParser()
answer = chain.invoke({"instr": instr, "context": context}).split("\n")
print(answer)

['1. Understand the definition of a subspace.', '2. Understand the definition of a linear transformation.', '3. Apply the definition of a linear transformation to a subspace.', '4. Check if the result of the transformation is still a subspace.', '5. Compare the result with the original subspace.', '6. Determine if the transformation preserves the subspace property.', '7. Conclude whether the statement "Linear transformation of the subspace is also subspace" is true or false based on the results.']


In [53]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
    ]
)

chain = rag_with_history_prompt | llm | StrOutputParser()

history = []

for i in range(len(answer)):
    if (i == 0):
        history.append(
            HumanMessage(answer[i] + "\n Refer to or use this context.\n Context:" + context)
        )
    else:
        history.append(
            HumanMessage(answer[i]) 
        )
    chain_result = chain.invoke({"history": history})
    history.append(
        AIMessage(chain_result)
    )

history.append(
    HumanMessage(instr)
)
answer = chain_result = chain.invoke({"history": history})
print(answer)


To prove or disprove the statement "Linear transformation of the subspace is also subspace," we need to consider the properties of linear transformations and subspaces.

A linear transformation T: V → W, where V and W are vector spaces over the same field F, is a function that satisfies two properties:

1. Homogeneity: T(αv) = αT(v) for all α ∈ F and v ∈ V.
2. Additivity: T(v1 + v2) = T(v1) + T(v2) for all v1, v2 ∈ V.

A subspace U of a vector space V is a subset of V that satisfies three properties:

1. Closure under vector addition: For all u1, u2 ∈ U, u1 + u2 ∈ U.
2. Closure under scalar multiplication: For all α ∈ F and u ∈ U, αu ∈ U.
3. Contains the zero vector of V: There exists a vector 0 ∈ U such that for any u ∈ U, u + 0 = u.

Now, let's consider the image of a subspace U under a linear transformation T: T(U) = {T(u) : u ∈ U}.

1. Closure under vector addition: Since T is additive, T(u1 + u2) = T(u1) + T(u2) for all u1, u2 ∈ U. This means that the image of the sum of two vecto